In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import matplotlib.pyplot as plt

from tqdm.auto import tqdm
import glob

import utils

In [3]:
from ge_bc import *

In [197]:
env_ids = 'Adventure AirRaid Alien Amidar Asterix Atlantis Atlantis2 Backgammon BankHeist BasicMath BattleZone BeamRider Berzerk Blackjack Breakout Carnival Casino Centipede ChopperCommand CrazyClimber Crossbow Darkchambers Defender DemonAttack DonkeyKong DoubleDunk Earthworld ElevatorAction Enduro Entombed Et FishingDerby Freeway Gopher Gravitar Hangman HauntedHouse Hero HumanCannonball Jamesbond JourneyEscape Kaboom KeystoneKapers Klax Koolaid Krull LaserGates MarioBros MiniatureGolf MontezumaRevenge MrDo Othello Phoenix Pitfall Pitfall2 Pong Pooyan PrivateEye Qbert Riverraid Robotank Seaquest SirLancelot SpaceInvaders SpaceWar Superman Tennis Tetris TicTacToe3d TimePilot Trondead Turmoil Tutankham UpNDown Venture VideoCheckers VideoChess VideoCube VideoPinball WizardOfWor WordZapper YarsRevenge Zaxxon'

env_ids = env_ids.split()
env_ids = env_ids[:]
env_ids = ' '.join(env_ids)
env_ids


'Adventure AirRaid Alien Amidar Asterix Atlantis Atlantis2 Backgammon BankHeist BasicMath BattleZone BeamRider Berzerk Blackjack Breakout Carnival Casino Centipede ChopperCommand CrazyClimber Crossbow Darkchambers Defender DemonAttack DonkeyKong DoubleDunk Earthworld ElevatorAction Enduro Entombed Et FishingDerby Freeway Gopher Gravitar Hangman HauntedHouse Hero HumanCannonball Jamesbond JourneyEscape Kaboom KeystoneKapers Klax Koolaid Krull LaserGates MarioBros MiniatureGolf MontezumaRevenge MrDo Othello Phoenix Pitfall Pitfall2 Pong Pooyan PrivateEye Qbert Riverraid Robotank Seaquest SirLancelot SpaceInvaders SpaceWar Superman Tennis Tetris TicTacToe3d TimePilot Trondead Turmoil Tutankham UpNDown Venture VideoCheckers VideoChess VideoCube VideoPinball WizardOfWor WordZapper YarsRevenge Zaxxon'

In [198]:
args = f'--env_ids {env_ids}'
args += ' --n_archives=10 --ge_data_dir=./data/egb_goexplore/ --n_envs=4 --n_steps=512'
args = parse_args(args.split())
args

Namespace(track=False, entity=None, project=None, name=None, device='cpu', seed=0, model='cnn_4', load_ckpt=None, save_ckpt=None, n_ckpts=1, lr=0.00025, clip_grad_norm=1.0, env_ids=['Adventure', 'AirRaid', 'Alien', 'Amidar', 'Asterix', 'Atlantis', 'Atlantis2', 'Backgammon', 'BankHeist', 'BasicMath', 'BattleZone', 'BeamRider', 'Berzerk', 'Blackjack', 'Breakout', 'Carnival', 'Casino', 'Centipede', 'ChopperCommand', 'CrazyClimber', 'Crossbow', 'Darkchambers', 'Defender', 'DemonAttack', 'DonkeyKong', 'DoubleDunk', 'Earthworld', 'ElevatorAction', 'Enduro', 'Entombed', 'Et', 'FishingDerby', 'Freeway', 'Gopher', 'Gravitar', 'Hangman', 'HauntedHouse', 'Hero', 'HumanCannonball', 'Jamesbond', 'JourneyEscape', 'Kaboom', 'KeystoneKapers', 'Klax', 'Koolaid', 'Krull', 'LaserGates', 'MarioBros', 'MiniatureGolf', 'MontezumaRevenge', 'MrDo', 'Othello', 'Phoenix', 'Pitfall', 'Pitfall2', 'Pong', 'Pooyan', 'PrivateEye', 'Qbert', 'Riverraid', 'Robotank', 'Seaquest', 'SirLancelot', 'SpaceInvaders', 'SpaceWa

In [199]:
np.random.seed(1)

In [200]:
print("Loading archives and Creating Trajs")
env_id2trajs = load_env_id2trajs(args.env_ids, args.ge_data_dir, args.n_archives, strategy=args.strategy, min_traj_len=args.min_traj_len)
for env_id, trajs in env_id2trajs.items():
    print(f"env_id: {env_id}, #trajs: {len(trajs)}")

def sample_traj_fn(id):
    env_id = args.env_ids[id // args.n_envs]
    trajs = env_id2trajs[env_id]
    return trajs[np.random.choice(len(trajs))]

Loading archives and Creating Trajs


  0%|          | 0/83 [00:00<?, ?it/s]

env_id: Adventure, #trajs: 10
env_id: AirRaid, #trajs: 10
env_id: Alien, #trajs: 10
env_id: Amidar, #trajs: 10
env_id: Asterix, #trajs: 10
env_id: Atlantis, #trajs: 10
env_id: Atlantis2, #trajs: 8
env_id: Backgammon, #trajs: 10
env_id: BankHeist, #trajs: 10
env_id: BasicMath, #trajs: 10
env_id: BattleZone, #trajs: 10
env_id: BeamRider, #trajs: 10
env_id: Berzerk, #trajs: 10
env_id: Blackjack, #trajs: 10
env_id: Breakout, #trajs: 10
env_id: Carnival, #trajs: 10
env_id: Casino, #trajs: 10
env_id: Centipede, #trajs: 10
env_id: ChopperCommand, #trajs: 10
env_id: CrazyClimber, #trajs: 10
env_id: Crossbow, #trajs: 10
env_id: Darkchambers, #trajs: 10
env_id: Defender, #trajs: 10
env_id: DemonAttack, #trajs: 10
env_id: DonkeyKong, #trajs: 10
env_id: DoubleDunk, #trajs: 10
env_id: Earthworld, #trajs: 10
env_id: ElevatorAction, #trajs: 10
env_id: Enduro, #trajs: 10
env_id: Entombed, #trajs: 10
env_id: Et, #trajs: 10
env_id: FishingDerby, #trajs: 10
env_id: Freeway, #trajs: 10
env_id: Gopher, #tr

In [201]:
env = make_env(args)

  0%|          | 0/83 [00:00<?, ?it/s]

In [202]:
buf = GEBuffer(env, args.n_steps, sample_traj_fn=sample_traj_fn, device=args.device)

In [203]:
for i in tqdm(range(4)):
    buf.collect()

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/512 [00:00<?, ?it/s]

  0%|          | 0/512 [00:00<?, ?it/s]

  0%|          | 0/512 [00:00<?, ?it/s]

  0%|          | 0/512 [00:00<?, ?it/s]

In [195]:
len(args.env_ids)*args.n_envs*args.n_steps*10/70

1517.7142857142858